In [1]:
from toxic.model import get_model
from toxic.nltk_utils import tokenize_sentences
from toxic.train_utils import train_folds
from toxic.embedding_utils import read_embedding_list, clear_embedding_list, convert_tokens_to_ids

import argparse
import numpy as np
import os
import pandas as pd

Using TensorFlow backend.


In [2]:
UNKNOWN_WORD = "_UNK_"
END_WORD = "_END_"
NAN_WORD = "_NAN_"

CLASSES = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

PROBABILITIES_NORMALIZE_COEFFICIENT = 1.4

In [3]:
train_file_path = "/Users/sdivakarla/bbanalytics-telemetry-research/satish/SentimentAnalysis/data/WikiToxicity/KaggleToxicDataset/train.csv"
test_file_path = "/Users/sdivakarla/bbanalytics-telemetry-research/satish/SentimentAnalysis/data/WikiToxicity/KaggleToxicDataset/test.csv"
embedding_path = "/Users/sdivakarla/bbanalytics-telemetry-research/satish/SentimentAnalysis/model/crawl/crawl-300d-2M.vec"
print("Loading data...")
train_data = pd.read_csv(train_file_path)
test_data = pd.read_csv(test_file_path)

list_sentences_train = train_data["comment_text"].fillna(NAN_WORD).values
list_sentences_test = test_data["comment_text"].fillna(NAN_WORD).values
y_train = train_data[CLASSES].values

Loading data...


In [4]:
print("Tokenizing sentences in train set...")
tokenized_sentences_train, words_dict = tokenize_sentences(list_sentences_train, {})

print("Tokenizing sentences in test set...")
tokenized_sentences_test, words_dict = tokenize_sentences(list_sentences_test, words_dict)

  0%|          | 123/159571 [00:00<02:09, 1228.74it/s]

Tokenizing sentences in train set...


  0%|          | 293/153164 [00:00<01:44, 1458.94it/s]

Tokenizing sentences in test set...


100%|██████████| 153164/153164 [01:39<00:00, 1534.72it/s]


In [5]:
embedding_path = "/Users/sdivakarla/bbanalytics-telemetry-research/satish/SentimentAnalysis/model/crawl/crawl-300d-2M.vec"

In [6]:
print(embedding_path)
words_dict[UNKNOWN_WORD] = len(words_dict)

print("Loading embeddings...")
embedding_list, embedding_word_dict = read_embedding_list(file_path=embedding_path)
embedding_size = len(embedding_list[0])

/Users/sdivakarla/bbanalytics-telemetry-research/satish/SentimentAnalysis/model/crawl/crawl-300d-2M.vec
Loading embeddings...


100%|██████████| 1999999/1999999 [03:07<00:00, 10675.88it/s]


In [7]:
print("Preparing data...")
embedding_list, embedding_word_dict = clear_embedding_list(embedding_list, embedding_word_dict, words_dict)

embedding_word_dict[UNKNOWN_WORD] = len(embedding_word_dict)
embedding_list.append([0.] * embedding_size)
embedding_word_dict[END_WORD] = len(embedding_word_dict)
embedding_list.append([-1.] * embedding_size)

embedding_matrix = np.array(embedding_list)

Preparing data...


In [8]:
print(embedding_matrix.shape)

(170075, 300)


In [9]:
sentences_length = 500
result_path = "toxic_results"
batch_size = 256
sentences_length = 500
recurrent_units=64
dropout_rate = 0.3
dense_size=32
fold_count=10

id_to_word = dict((id, word) for word, id in words_dict.items())
train_list_of_token_ids = convert_tokens_to_ids(tokenized_sentences_train,id_to_word,
                                                embedding_word_dict,sentences_length)
test_list_of_token_ids = convert_tokens_to_ids(tokenized_sentences_test,id_to_word,
                                               embedding_word_dict,sentences_length)
X_train = np.array(train_list_of_token_ids)
X_test = np.array(test_list_of_token_ids)

In [ ]:
get_model_func = lambda: get_model(
    embedding_matrix,
    sentences_length,
    dropout_rate,
    recurrent_units,
    dense_size)

In [ ]:
print("Starting to train models...")
models = train_folds(X_train, y_train, fold_count, batch_size, get_model_func)

Starting to train models...
Epoch 1/1
143614/143614 [==============================] - 3288s 23ms/step - loss: 0.0648 - acc: 0.9770
Epoch 0 loss 0.04540272414923855 best_loss -1
Epoch 1/1
106752/143614 [=====================>........] - ETA: 15:03 - loss: 0.0445 - acc: 0.9830

In [ ]:
result_path = toxic_results
if not os.path.exists(result_path):
        os.mkdir(result_path)

In [ ]:
print("Predicting results...")
test_predicts_list = []
for fold_id, model in enumerate(models):
    model_path = os.path.join(args.result_path, "model{0}_weights.npy".format(fold_id))
    np.save(model_path, model.get_weights())
    test_predicts_path = os.path.join(args.result_path, "test_predicts{0}.npy".format(fold_id))
    test_predicts = model.predict(X_test, batch_size=args.batch_size)
    test_predicts_list.append(test_predicts)
    np.save(test_predicts_path, test_predicts)

In [ ]:
test_predicts = np.ones(test_predicts_list[0].shape)
for fold_predict in test_predicts_list:
    test_predicts *= fold_predict
    test_predicts **= (1. / len(test_predicts_list))
    test_predicts **= PROBABILITIES_NORMALIZE_COEFFICIENT
    
    test_ids = test_data["id"].values
    test_ids = test_ids.reshape((len(test_ids), 1))

    test_predicts = pd.DataFrame(data=test_predicts, columns=CLASSES)
    test_predicts["id"] = test_ids
    test_predicts = test_predicts[["id"] + CLASSES]
    submit_path = os.path.join(args.result_path, "submit")
    test_predicts.to_csv(submit_path, index=False)